In [2]:
import pandas as pd
import re
import gzip

from gene_id2symbol import id2name
from collections import Counter

# Virus-human interaction integration from five databases

## 1. Raw data extraction
### BioGRID

In [3]:
gene_A = []
gene_B = []
method = []
pubmed = []
interaction_type = []
taxid = []
with gzip.open(r'../data/VHI_from_databases/BioGRID_human.txt.gz', 'rt') as f:
    for row in f:
        if row.startswith("#ID"):
            pass
        else:
            col = row.split("\t")
            if col[9] == 'taxid:9606' and col[10] == 'taxid:9606':
                pass
            else:
                if col[9] == 'taxid:9606':
                    gene_A.append(id2name(col[0].replace('entrez gene/locuslink:', '')))
                    gene_B.append(col[1].replace('entrez gene/locuslink:', ''))
                    taxid.append(col[10])
                else:
                    gene_A.append(id2name(col[1].replace('entrez gene/locuslink:', '')))
                    gene_B.append(col[0].replace('entrez gene/locuslink:', ''))
                    taxid.append(col[9])
                method.append(col[6].replace('psi-mi:"', '').replace('"', ''))
                pubmed.append(col[8])
                interaction_type.append(col[11].replace('psi-mi:"', '').replace('"', ''))

biogrid = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B,
                         'interaction_detection_method': method,
                         'pubmed_id': pubmed,
                         'interaction_type': interaction_type,
                         'database': ['BioGRID'] * len(gene_A),
                         'taxid': taxid})

In [4]:
biogrid.shape

(37692, 7)

### IntAct

In [5]:
gene_A = []
gene_B = []
method = []
pubmed = []
interaction_type = []
taxid = []
species = []
with gzip.open(r'../data/VHI_from_databases/intact_human-others.txt.gz', 'rt') as f:
    for row in f:
        col = row.strip().split('\t')
        if col[4] == '-' or col[5] == '-':
            pass
        elif re.search(r'pubmed:unassigned[0-9]+', col[3]):  # pubmed:unassigned
            if re.search(r'taxid:9606', col[4]):
                gene_A.append(id2name(re.split('[:-]', col[0])[1]))
                gene_B.append(re.split('[:-]', col[0])[1])
                taxid.append(re.split("\(", col[5])[0])
            else:
                gene_A.append(id2name(re.split('[:-]', col[1])[1]))
                gene_B.append(re.split('[:-]', col[0])[1])
                taxid.append(re.split("\(", col[4])[0])
            method.append(col[2].replace('psi-mi:"', '').replace('"', ''))
            pubmed.append(re.split("\|", col[3])[1])
            interaction_type.append(col[6].replace('psi-mi:"', '').replace('"', ''))
        else:
            if re.search(r'taxid:9606', col[4]):
                gene_A.append(id2name(re.split('[:-]', col[0])[1]))
                gene_B.append(re.split('[:-]', col[0])[1])
                taxid.append(re.split("\(", col[5])[0])
            else:
                gene_A.append(id2name(re.split('[:-]', col[1])[1]))
                gene_B.append(re.split('[:-]', col[0])[1])
                taxid.append(re.split("\(", col[4])[0])
            method.append(col[2].replace('psi-mi:"', '').replace('"', ''))
            pubmed.append(re.search(r'(.*)(pubmed:[0-9]+)(\D*)(.*)', col[3]).group(2))
            interaction_type.append(col[6].replace('psi-mi:"', '').replace('"', ''))

with gzip.open(r'../data/VHI_from_databases/intact_virus.txt.gz', 'rt') as f:
    for row in f:
        if row.startswith("#ID"):
            pass
        else:
            col = row.strip().split('\t')
            if col[9] == '-' or col[10] == '-':
                pass
            elif re.search(r'pubmed:unassigned[0-9]+', col[8]):  # pubmed:unassigned
                if re.search(r'taxid:9606', col[9]):
                    gene_A.append(id2name(re.split('[:-]', col[0])[1]))
                    gene_B.append(re.split('[:-]', col[0])[1])
                    taxid.append(re.split("\(", col[10])[0])
                else:
                    gene_A.append(id2name(re.split('[:-]', col[1])[1]))
                    gene_B.append(re.split('[:-]', col[0])[1])
                    taxid.append(re.split("\(", col[9])[0])
                method.append(col[6].replace('psi-mi:"', '').replace('"', ''))
                pubmed.append('pubmed:unassigned')
                interaction_type.append(col[11].replace('psi-mi:"', '').replace('"', ''))
            else:
                if re.search(r'taxid:9606', col[9]):
                    gene_A.append(id2name(re.split('[:-]', col[0])[1]))
                    gene_B.append(re.split('[:-]', col[0])[1])
                    taxid.append(re.split("\(", col[10])[0])
                else:
                    gene_A.append(id2name(re.split('[:-]', col[1])[1]))
                    gene_B.append(re.split('[:-]', col[0])[1])
                    taxid.append(re.split("\(", col[9])[0])
                method.append(col[6].replace('psi-mi:"', '').replace('"', ''))
                pubmed.append(re.search(r'(.*)(pubmed:[0-9]+)(\D*)(.*)', col[8]).group(2))
                interaction_type.append(col[11].replace('psi-mi:"', '').replace('"', ''))

intact = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B,
                         'interaction_detection_method': method,
                         'pubmed_id': pubmed,
                         'interaction_type': interaction_type,
                         'database': ['IntAct'] * len(gene_A),
                         'taxid': taxid})

In [6]:
intact.shape

(103646, 7)

### MINT

In [7]:
gene_A = []
gene_B = []
method = []
pubmed = []
interaction_type = []
taxid = []
with gzip.open(r'../data/PPIs_from_databases/MINT_human.gz', 'rt') as f:
    for row in f:
        col = row.strip().split('\t')
        if re.search(r'taxid:9606', col[9]) and re.search(r'taxid:9606', col[10]):
            pass
        else:
            if col[9] == 'taxid:9606(human)|taxid:9606(Homo sapiens)':
                gene_A.append(id2name(re.split('[:-]', col[0])[1]))
                gene_B.append(re.split('[:-]', col[1])[1])
                taxid.append(re.split("\(", col[10])[0])
            else:
                gene_A.append(id2name(re.split('[:-]', col[1])[1]))
                gene_B.append(re.split('[:-]', col[0])[1])
                taxid.append(re.split("\(", col[9])[0])
            method.append(col[6].replace('psi-mi:"', '').replace('"', ''))
            pubmed.append(re.search(r'(.*)(pubmed:[0-9]+)(\D*)(.*)', col[8]).group(2))
            interaction_type.append(col[11].replace('psi-mi:"', '').replace('"', ''))

mint = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B,
                         'interaction_detection_method': method,
                         'pubmed_id': pubmed,
                         'interaction_type': interaction_type,
                         'database': ['MINT'] * len(gene_A),
                         'taxid': taxid})

In [8]:
mint.shape

(18225, 7)

### DIP

In [9]:
gene_A = []
gene_B = []
method = []
pubmed = []
interaction_type = []
taxid = []
with gzip.open(r'../data/PPIs_from_databases/Hsapi20170205.txt.gz', 'rt') as f:
    for row in f:
        col = row.strip().split("\t")
        if len(col) < 15:
            pass
        else:
            if col[9] == 'taxid:9606(Homo sapiens)' and col[10] == 'taxid:9606(Homo sapiens)':
                pass
            elif bool(re.search("uniprotkb", col[0])) and bool(re.search("uniprotkb", col[1])):
                if col[9] == 'taxid:9606(Homo sapiens)':
                    gene_A.append(id2name(re.split('uniprotkb:', col[0])[1]))
                    gene_B.append(re.split('uniprotkb:', col[1])[1])
                    taxid.append(re.split("\(", col[10])[0])
                else:
                    gene_A.append(id2name(re.split('uniprotkb:', col[1])[1]))
                    gene_B.append(re.split('uniprotkb:', col[0])[1])
                    taxid.append(re.split("\(", col[9])[0])
                method.append(col[6].replace('psi-mi:"', '').replace('"', ''))
                pubmed.append(re.sub(r'\|pubmed:DIP-[0-9]+', '', col[8]))
                interaction_type.append(col[11])


dip = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B,
                      'interaction_detection_method': method,
                      'pubmed_id': pubmed,
                      'interaction_type': interaction_type,
                      'database': ['DIP'] * len(gene_A),
                      'taxid': taxid})

dip2 = dip[['gene_A', 'interaction_detection_method']]
dip2 = dip2.drop('interaction_detection_method', axis=1).\
    join(dip2['interaction_detection_method'].str.split('|', expand = True).\
         stack().reset_index(level=1, drop=True).rename('interaction_detection_method'))

dip3 = dip[['gene_B', 'pubmed_id']]
dip3 = dip3.drop('pubmed_id', axis=1).\
    join(dip3['pubmed_id'].str.split('|', expand = True).\
         stack().reset_index(level=1, drop=True).rename('pubmed_id'))

dip4 = dip[['interaction_type', 'database', 'taxid']]
dip4 = dip4.drop('interaction_type', axis=1).\
    join(dip4['interaction_type'].str.split('|', expand = True).\
         stack().reset_index(level=1, drop=True).rename('interaction_type'))

dip5 = pd.concat([dip2, dip3, dip4], axis=1)
dip5 = dip5[['gene_A', 'gene_B', 'interaction_detection_method', 'pubmed_id',
             'interaction_type', 'database', 'taxid']]

In [10]:
dip5.shape

(1061, 7)

### VirHostNet

In [11]:
gene_A = []
gene_B = []
method = []
pubmed = []
interaction_type = []
taxid = []
with gzip.open(r'../data/VHI_from_databases/virhostnet.txt.gz', 'rt') as f:
    for row in f:
        col = row.strip().split("\t")
        if col[9] == col[10]:
            pass
        elif col[9] == 'taxid:9606' or col[10] == 'taxid:9606':
            if col[9] == 'taxid:9606':
                gene_A.append(id2name(re.split('[:-]', col[0])[1]))
                gene_B.append(re.split('[:-]', col[1])[1])
                taxid.append(col[10])
            else:
                gene_A.append(id2name(re.split('[:-]', col[1])[1]))
                gene_B.append(re.split('[:-]', col[0])[1])
                taxid.append(col[9])
            method.append(col[6].replace('psi-mi:"', '').replace('"', ''))
            pubmed.append(col[8])
            interaction_type.append(col[11].replace('psi-mi:"', '').replace('"', ''))

virhostnet = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B,
                         'interaction_detection_method': method,
                         'pubmed_id': pubmed,
                         'interaction_type': interaction_type,
                         'database': ['VirHostNet'] * len(gene_A),
                         'taxid': taxid})

In [12]:
virhostnet.shape

(33300, 7)

### HPIDB

In [13]:
gene_A = []
gene_B = []
method = []
pubmed = []
interaction_type = []
taxid = []
with gzip.open(r'../data/VHI_from_databases/hpidb2.mitab.txt.gz', 'rt') as f:
    for row in f:
        if row.startswith("# protein_xref_1"):
            pass
        else:
            col = row.strip().split('\t')
            if re.search(r'pubmed:unassigned[0-9]+', col[3]):  # pubmed:unassigned
                if re.search(r'taxid:9606', col[4]):
                    gene_A.append(id2name(re.split('-', re.split(':', col[0])[-1])[0]))
                    gene_B.append(re.split('-', re.split(':', col[1])[-1])[0])
                    taxid.append(re.split("\(", col[5])[0])
                else:
                    gene_A.append(id2name(re.split('-', re.split(':', col[1])[-1])[0]))
                    gene_B.append(re.split('-', re.split(':', col[0])[-1])[0])
                    taxid.append(re.split("\(", col[4])[0])
                method.append(col[2].replace('psi-mi:', ''))
                pubmed.append(re.split("\|", col[3])[1])
                interaction_type.append(col[6].replace('psi-mi:', ''))
            else:
                if re.search(r'taxid:9606', col[4]):
                    gene_A.append(id2name(re.split('-', re.split(':', col[0])[-1])[0]))
                    gene_B.append(re.split('-', re.split(':', col[1])[-1])[0])
                    taxid.append(re.split("\(", col[5])[0])
                else:
                    gene_A.append(id2name(re.split('-', re.split(':', col[1])[-1])[0]))
                    gene_B.append(re.split('-', re.split(':', col[0])[-1])[0])
                    taxid.append(re.split("\(", col[4])[0])
                method.append(col[2].replace('psi-mi:', ''))
                pubmed.append(re.search(r'(.*)(pubmed:[0-9]+)(\D*)(.*)', col[3]).group(2))
                interaction_type.append(col[6].replace('psi-mi:', ''))


hpidb = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B,
                         'interaction_detection_method': method,
                         'pubmed_id': pubmed,
                         'interaction_type': interaction_type,
                         'database': ['HPIDB'] * len(gene_A),
                         'taxid': taxid})

In [14]:
hpidb.shape

(69787, 7)

## 2. Intrgration

In [15]:
vhi = pd.concat([biogrid, intact, mint, dip5, virhostnet, hpidb])
vhi2 = vhi[vhi.gene_A != 0]
vhi2 = vhi2.drop_duplicates()

In [16]:
vhi2.shape

(203919, 7)

In [17]:
"""Remove invalid methods, genetic interactions and PTMs"""
vhi3 = vhi2[(vhi2.interaction_type == 'MI:0218(physical interaction)') |
            (vhi2.interaction_type == 'MI:0407(direct interaction)') |
            (vhi2.interaction_type == 'MI:0914(association)') |
            (vhi2.interaction_type == 'MI:0915(physical association)')]

vhi3 = vhi3.drop(['interaction_type'], axis=1)
vhi3 = vhi3.drop_duplicates()

In [18]:
vhi3.shape

(197256, 6)

In [19]:
"""Virus description"""
virus_description = pd.read_csv(r'../data/virus_description.txt',
                    sep='\t')
virus_taxid = ['taxid:' + i for i in virus_description['Taxid'].apply(str).tolist()]
vhi4 = vhi3[vhi3['taxid'].isin(virus_taxid)]

In [20]:
vhi4.shape

(104359, 6)

In [21]:
"""Merge evidence have an ancestor-descendent relationship"""
method_category = pd.read_csv(r'../data/Interaction_detection_method.txt',
                              sep="\t")
method_category = method_category[['Interaction Detection Method', 'Binary call', 'Parent MI']]
method_category.columns = ['interaction_detection_method', 'binary_call', 'parent_mi']

vhi5 = pd.merge(vhi4, method_category, how='inner',
                on='interaction_detection_method')
vhi5 = vhi5[vhi5['binary_call'] != 'invalid']
vhi5 = vhi5.drop(['interaction_detection_method','binary_call'], axis=1)
vhi5 = vhi5.drop_duplicates()

In [22]:
vhi5.shape

(95632, 6)

In [23]:
"""Combine the same interaction from different literature;
Combine the same interaction from different experimental scale;
Combine the same interaction from different database;
Join the same interaction divided into different method category;
Join the same interaction used different method (MI-PSI)"""

def comb(df):
    return ';'.join('%s' %i for i in df.values)

def inter_comb(name):
    ppi = vhi5[['gene_A','gene_B', 'taxid', name]]
    ppi = ppi.drop_duplicates()
    ppi2 = ppi.groupby(['gene_A','gene_B', 'taxid'])[name].apply(comb)
    interaction_list = []
    for i in range(len(ppi2.index)):
        interaction_list.append(ppi2.index[i])
    ppi3 = pd.DataFrame(interaction_list)
    ppi3.columns = ppi2.index.names
    ppi3[name] = ppi2.values
    return ppi3

vhi6 = pd.DataFrame({'gene_A':[], 'gene_B':[], 'taxid':[]})
for name in ['pubmed_id', 'database', 'parent_mi']:
    ppi = inter_comb(name)
    vhi6 = pd.merge(vhi6, ppi, how='right',
                    on=['gene_A', 'gene_B', 'taxid'])

vhi6 = vhi6[vhi6['gene_B']!='EBI']
vhi6 = vhi6[vhi6['pubmed_id']!='pubmed:unassigned']

virus_description['Taxid'] = virus_taxid
vhi7 = pd.merge(vhi6, virus_description, how='left',
                left_on='taxid', right_on='Taxid')

In [24]:
vhi7.shape

(42166, 12)

In [25]:
"""ID mapping to UniProtKB id"""
ID2uniprot = pd.read_csv(r'../data/ID2uniprot.txt',
                         sep='\t', header=None)
ID2uniprot.columns = ['gene_B', 'uniprot']

vhi8 = pd.merge(vhi7, ID2uniprot, how='inner', on='gene_B')
vhi8 = vhi8.drop(['gene_B'], axis=1)
vhi8 = vhi8.drop_duplicates()

gene_B = vhi8.uniprot
vhi8 = vhi8.drop(['uniprot'], axis=1)
vhi8.insert(1, 'gene_B', gene_B)
vhi8 = vhi8.drop('taxid', axis=1)

vtgs = vhi8[['gene_A', 'Parent Viral Name', 'Baltimore', 'Family']]
vtgs = vtgs.drop_duplicates()
vtgs.columns = ['Gene', 'Virus', 'Baltimore', 'Family']

In [26]:
vtgs.head()

,Gene,Virus,Baltimore,Family
0,A2M,Human cytomegalovirus,dsDNA,Herpesviridae
1,A2ML1,Human cytomegalovirus,dsDNA,Herpesviridae
2,ACAA1,Human cytomegalovirus,dsDNA,Herpesviridae
3,ACOX1,Human cytomegalovirus,dsDNA,Herpesviridae
4,ACP7,Human cytomegalovirus,dsDNA,Herpesviridae


In [27]:
vtgs.shape

(21455, 4)

In [28]:
"""Targets by viruses, except coronavirus"""
vtgs = vtgs[(vtgs.Virus != "SARS-CoV-1")]
vtgs = vtgs[(vtgs.Virus != "HCoV-229E")]
vtgs = vtgs[(vtgs.Virus != "HCoV-NL63")]
vtgs = vtgs[(vtgs.Virus != "HCoV-OC43")]

In [29]:
vtgs.shape

(21366, 4)

## 3. Coronavirus-human PPI from literatures

In [30]:
bait = []
prey = []
virus = []
pubmed_id = []

# from Gordon et al. Science 2020
for cov in ['MERS-CoV', 'SARS-CoV-1', "SARS-CoV-2"]:
    with open("../data/VHI_from_databases/From_literatures/Science2020/" + cov + "-human.txt") as f:
        for row in f:
            if row.startswith("Bait"):
                pass
            else:
                col = row.strip().split('\t')
                bait.append(col[0].upper())
                prey.append(col[2])
                virus.append(cov)
                pubmed_id.append("pubmed:33060197")
                
# from Stukalov et al., Nature 2021
for cov in ['HCoV-229E', 'HCoV-NL63', 'SARS-CoV-1', 'SARS-CoV-2']:
    with open("../data/VHI_from_databases/From_literatures/Nature2021/" + cov + "-human.txt") as f:
        for row in f:
            if row.startswith("Bait"):
                pass
            else:
                col = row.strip().split('\t')
                bait.append(col[0].upper())
                prey.append(col[1])
                virus.append(cov)
                pubmed_id.append("pubmed:33845483")
                
# from Li et al., Med 2021
with open(r'../data/VHI_from_databases/From_literatures/Med2021/SARS-CoV-2-human.txt') as f:
    for row in f:
        if row.startswith("Bait"):
            pass
        else:
            col = row.strip().split('\t')
            bait.append(col[0].upper())
            prey.append(col[1])
            virus.append(cov)
            pubmed_id.append("pubmed:32838362")

In [31]:
"""Data integration"""
cov_h = pd.DataFrame({'Bait': bait, 'PreyGene': prey, 'Virus': virus, 'Pubmed_id': pubmed_id})

def comb(df):
    return ';'.join(df.values)

cov_h2 = cov_h.groupby(['Bait', 'PreyGene', 'Virus'])['Pubmed_id'].apply(comb)
cov_ppis = []
for i in range(len(cov_h2.index)):
    cov_ppis.append(cov_h2.index[i])
cov_h3 = pd.DataFrame(cov_ppis)
cov_h3.columns = cov_h2.index.names
cov_h3['Pubmed_id'] = cov_h2.values

cov_h3.to_csv(path_or_buf='../intermediate/CoVs-human_PPI.txt', index = False, sep = '\t')

In [32]:
cov_h.head()

,Bait,PreyGene,Virus,Pubmed_id
0,E,SPTLC2,MERS-CoV,pubmed:33060197
1,E,B4GAT1,MERS-CoV,pubmed:33060197
2,E,ATP6AP2,MERS-CoV,pubmed:33060197
3,E,IGHG4,MERS-CoV,pubmed:33060197
4,E,WLS,MERS-CoV,pubmed:33060197


## 4. All the VTGs

In [33]:
cov_h4 = cov_h3[['PreyGene', 'Virus']]
cov_h4.columns = ['Gene', 'Virus']
cov_h4 = cov_h4.drop_duplicates()
cov_h4['Baltimore'] = ['(+)ssRNA'] * cov_h4.shape[0]
cov_h4['Family'] = ['Coronaviridae'] * cov_h4.shape[0]

In [34]:
cov_h4.shape

(2759, 4)

In [35]:
vtgs = pd.concat([vtgs, cov_h4], axis = 0, ignore_index=True)
vtgs = vtgs.drop_duplicates()

In [36]:
vtgs.head()

,Gene,Virus,Baltimore,Family
0,A2M,Human cytomegalovirus,dsDNA,Herpesviridae
1,A2ML1,Human cytomegalovirus,dsDNA,Herpesviridae
2,ACAA1,Human cytomegalovirus,dsDNA,Herpesviridae
3,ACOX1,Human cytomegalovirus,dsDNA,Herpesviridae
4,ACP7,Human cytomegalovirus,dsDNA,Herpesviridae


In [37]:
vtgs.shape

(23832, 4)

## 5. Virally-targeted modules (no statistical test)

In [40]:
cluster_genes = {}
i = 1
with open(r'../intermediate/batch_MCL_out/out.HUMPPI2022.I228') as f:
    for row in f:
        cols = row.strip().split('\t')
        if len(cols) > 2:
            cluster_genes[i] = cols
            i = i + 1

def geneincluster(x):
    for k in cluster_genes.keys():
        if x in cluster_genes[k]:
            return k

In [41]:
geneincluster('A2M')

856

In [42]:
vtgs['Module'] = vtgs['Gene'].apply(geneincluster)
vtgs = vtgs.where(vtgs.notnull(), 0)
vtgs['Module'] = vtgs['Module'].apply(int)

vtgs.to_csv(path_or_buf='../intermediate/virally-targeted_genes.txt', index = False, sep = '\t')